# Supratidal and coastal floodplain forests workflow
- for each tile
- generates elevation
- generates connectivity
- combines to generate tidal connectivity layer (TCM)
- generates woody cover
- output all 4 layers (elevation, connectivity, veg (woody cover), SCFF extent)

Code taken from   
https://github.com/christopherowers/supratidal_forests/blob/SCFF_workflow/notebooks/SCF_elevation.ipynb
https://github.com/christopherowers/supratidal_forests/blob/SCFF_workflow/notebooks/SCF_connectivity.ipynb
https://github.com/christopherowers/supratidal_forests/blob/SCFF_workflow/notebooks/SCF_woody_model.ipynb
https://github.com/christopherowers/supratidal_forests/blob/SCFF_timeseries/notebooks/SCFF_timeseries_loop.ipynb

In [1]:
import time
start = time.time()

In [2]:
import sys

from odc.geo.gridspec import GridSpec
from odc.geo.geobox import GeoBox
from odc.geo.geom import BoundingBox
from odc.geo.types import xy_

import numpy as np
import xarray as xr
import geopandas as gpd

import datacube
dc = datacube.Datacube()

from dea_tools.spatial import xr_interpolate
from dea_tools.datahandling import load_reproject

sys.path.insert(1, "/home/jovyan/code/supratidal_forests/data")
from gridspec_96km_tiles import tiles

sys.path.insert(2, "/home/jovyan/code/supratidal_forests/functions")
from least_cost_path import xr_cost_distance, load_connectivity_mask_aquatic

# load virtual product
import importlib
from datacube.virtual import catalog_from_file
from datacube.virtual import DEFAULT_RESOLVER
catalog = catalog_from_file('../virtual_products/virtual_product_cat_wcf.yaml')

### setup gridspec

In [3]:
# Expanded Extents 96 km tiling grid
gs_c3_expanded = GridSpec(
    crs="EPSG:3577",
    resolution=30,
    tile_shape=(3200, 3200),
    origin=xy_(-4416000, -6912000),
)

In [4]:
# Visualising whole grid
# Set bounding box
c3_bbox = BoundingBox(
    left=-2400000, bottom=-5088000, right=2784000, top=-864000, crs="EPSG:3577"
)

# Convert grid to geopandas polygon dataset covering full C3 grid extent
grid_gdf = gpd.GeoDataFrame.from_features(
    gs_c3_expanded.geojson(bbox=c3_bbox),
    crs="EPSG:4326", # CRS different here just for visualisation
)
grid_gdf.explore()

### load HAT and SS

In [5]:
# load in file
HAT_gpd = gpd.read_file('/home/jovyan/gdata1/projects/coastal/supratidal_forests/data/HAT_file.gpkg') # TODO: Raf, change this back to original file from Paul B - this has x and y, whereas the modified version has labels of lat long (but they are not)

# Converting HAT file to CRS 3577 to match gridspec
HAT_gpd_EPSG3577 = HAT_gpd.to_crs("EPSG:3577")

In [6]:
# load in file
SS_gpd = gpd.read_file('/home/jovyan/gdata1/projects/coastal/supratidal_forests/data/STF_SS_ElevationClasses.geojson')

# Converting SS file to CRS 3577 to match gridspec
SS_gpd_EPSG3577 = SS_gpd.to_crs("EPSG:3577")

In [7]:
# TODO: Raf, why is this diffferent to previous code? - Raf to add comment
# HAT_SS_gpd = gpd.sjoin(HAT_gpd_EPSG3577, SS_gpd_EPSG3577, predicate='within')
HAT_SS_gpd = gpd.overlay(HAT_gpd_EPSG3577, SS_gpd_EPSG3577, how='intersection')

In [8]:
# creating a new column in mainland_grid and populating with the sum of HAT and Storm Surge
HAT_SS_gpd['HATSS'] = HAT_SS_gpd['HAT'] + HAT_SS_gpd['SSElev']

### Set up woody cover model

In [9]:
# load in vp woody cover
sys.path.insert(1, '../virtual_products') 
# The dictionary which datacube uses to understand (resolve) the different virtual product functionality
# Need to add any aggregations for the VP you're using

# Get location of transformation
aggregation = "best_pixel_gmad"
agg_loc = importlib.import_module(aggregation)
agg_class = aggregation.split('.')[-1]

DEFAULT_RESOLVER.register('aggregate', agg_class, getattr(agg_loc, agg_class) )

# Need to add any tranformations for the VP you're using
# Get location of transformation
transformation = "woody_cover"
trans_loc = importlib.import_module(transformation)
trans_class = transformation.split('.')[-1]

DEFAULT_RESOLVER.register('transform', trans_class, getattr(trans_loc, trans_class) )

### Set up any other functions required

In [10]:
# Define the exponential function for connectivity y = e^(-0.004 * x)
def apply_exponential(arr):
    return np.exp(-0.004 * arr)

### Select subset of tiles to run from gridspec .py file

In [11]:
select_tiles = tiles[262:264]
select_tiles

[(58, 23), (59, 23)]

### loop through tiles

In [12]:
# Loop through the list
for item in select_tiles:
    print(item)
    geobox = gs_c3_expanded[item]

    ### Bespoke setup's needed for models ###
    # Buffer input geobox a for connectivity (to reduce edge effects)
    buffer = 20000 # this is from load_connectivity_mask function. hardcoded here and will need to change if function value is change in load_connectivity_mask_aquatic
    geobox_buffered = GeoBox.from_bbox(
        geobox.buffered(xbuff=buffer, ybuff=buffer).boundingbox,
        resolution=30,
        tight=True,
    )

    # Load in water from wofs, as needed for woody cover bounds as well as used later for woody model aquatic mask
    wofs = dc.load(product="ga_ls_wo_fq_cyear_3", measurements=["frequency"], time = ("2020", "2020"), like=geobox)
    # get water class
    water = xr.where((wofs.frequency >= 0.2), 1, 0).astype('int8')
    
    # NOTE: bug in using geobox with virtual products. does not work even with odc.geobox.compat
    # working as below for woody cover vp, but very clunky way of getting this to work
    # Extract bounds for x and y as well as time from the dataset
    x_bounds = (wofs.x.min().item(), wofs.x.max().item())
    y_bounds = (wofs.y.min().item(), wofs.y.max().item())
    time_bounds = (str(wofs.time.dt.year[0].item()), str(wofs.time.dt.year[-1].item()))
    # Construct the query dictionary with CRS
    query = {
        'x': x_bounds,
        'y': y_bounds,
        'time': time_bounds,
        'crs': 'EPSG:3577'
    }
    
    ### Load other data products ###
    # Load srtm to geobox
    srtm_ds = dc.load(product = 'ga_srtm_dem1sv1_0', resampling="bilinear", like=geobox).squeeze('time')
    srtm = srtm_ds.dem_h
    # Load srtm to buffered geobox
    srtm_ds = dc.load(product = 'ga_srtm_dem1sv1_0', resampling="bilinear", like=geobox_buffered).squeeze('time')
    srtm_buffered = srtm_ds.dem_h
    
    # load in DEA mangroves
    DEAmangrove = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', time = ("2020", "2020"), like=geobox)
    # if no mangroves within AOI, create dummy xr.dataarray
    if DEAmangrove.data_vars == {}:
        mangrove = xr.DataArray(np.zeros_like(water), coords=water.coords, dims=water.dims, attrs=water.attrs)
    else:
        # get output of mangrove == 1, not mangrove == 0
        mangrove = (DEAmangrove.canopy_cover_class != 255).squeeze('time')
    # load in DEA mangroves buffered
    DEAmangrove = dc.load(product = 'ga_ls_mangrove_cover_cyear_3', time = ("2020", "2020"), like=geobox_buffered)
    # if no mangroves within AOI, create dummy xr.dataarray
    if DEAmangrove.data_vars == {}:
        mangrove_buffered = xr.DataArray(np.zeros_like(srtm_buffered), coords=srtm_buffered.coords, dims=srtm_buffered.dims, attrs=srtm_buffered.attrs)
    else:
        # get output of mangrove == 1, not mangrove == 0
        mangrove_buffered = (DEAmangrove.canopy_cover_class != 255).squeeze('time')

    # Load in JCU saltmarsh
    geotiff_path = '/home/jovyan/gdata1/data/saltmarsh/JCU_Australia-saltmarsh-extent_v1-0.tif'
    saltmarsh = load_reproject(path=geotiff_path, how=geobox)
    saltmarsh_buffered = load_reproject(path=geotiff_path, how=geobox_buffered)
    
    # Load in JCU saltflat
    geotiff_path = '/home/jovyan/gdata1/data/saltmarsh/JCU_Australia-saltflat-extent_v1-0.tif'
    saltflat = load_reproject(path=geotiff_path, how=geobox)
    saltflat_buffered = load_reproject(path=geotiff_path, how=geobox_buffered)
    
    # Load in CEM data 
    geotiff_path = '/home/jovyan/gdata1/projects/coastal/supratidal_forests/data/CEM_v002/ga_s2ls_coastalecosystems_cyear_3_2021--P1Y_interim_classification_0-0-2.tif'
    
    # CEM for veg model
    CEM_data = load_reproject(path=geotiff_path, how=geobox)
    CEM = ~CEM_data.isnull()

    # CEM for connectivity model (saltmarsh intersection)
    CEM_data = load_reproject(path=geotiff_path, how=geobox_buffered)
    CEM_layers = CEM_data.where((CEM_data == 2) | (CEM_data == 3) |(CEM_data == 5)) # all layers except saltmarsh
    CEM_buffered = ~CEM_layers.isnull()
    CEM_buffered_saltmarsh = CEM_data.where(CEM_data == 4) # saltmarsh in CEM equivalent to 4

    # Intersect both CEM and JCU saltmarsh layers based on common coordinates (inner join)
    da_intersected = xr.align(CEM_buffered_saltmarsh, saltmarsh_buffered, join='inner')
    # Extract the aligned DataArrays (now they share the same coordinates) - TODO: Raf, haven't they always had the same coords? they load in the same geobox
    CEM_buffered_saltmarsh_aligned, saltmarsh_buffered_aligned = da_intersected

    # Create a new DataArray based on the intersection (values aligned)
    saltmarsh_buffered_intersection = CEM_buffered_saltmarsh_aligned + saltmarsh_buffered_aligned
    saltmarsh_buffered_inter = ~saltmarsh_buffered_intersection.isnull()
    
    ### Elevation model ###
    # greater than -6m AHD and less than 11m AHD == True
    # some areas in NT are below 0 AHD and need to be included in potential supratidal extent, hence value of -6 that Raf has checked is sensible.
    # 11m max elevation now being used (was 10m previously) due to HAT + SS value max of 10.53m
    # for connectivity model less than 11m AHD == True (this needs to be thresholded as minimum at 0 for STF extent product due to supratidal areas not being below 0 AHD
    AHD_min = -6
    AHD_max = 11
    lessthan_AHD = srtm <= AHD_max
    greaterthan_AHD = srtm >= AHD_min
    srtm_mask = lessthan_AHD & greaterthan_AHD

    srtm_mask = srtm_mask.astype('int8')
    
    # get elevation values for srtm_mask
    supratidal_elev = srtm * srtm_mask
    supratidal_elev = xr.where(supratidal_elev == 0, np.nan, supratidal_elev.values)
    
    # interpolate HAT and SS using srtm bounds
    interpolate_idw = xr_interpolate(ds=srtm, gdf=HAT_SS_gpd, columns=['HAT', 'SSElev'], method="idw")
    HAT_interpolate = interpolate_idw.HAT
    SS_interpolate = interpolate_idw.SSElev
    
    # add two layers together
    HATSS_interpolate = HAT_interpolate + SS_interpolate

    # generate elevation probability product
    # values of 1 for <= HAT
    # values normalised between 1 and 0.5 > HAT and <= HAT_SS
    # values normalised between 0.5 and 1 > HAT_SS and <= 10m AHD
    HAT = xr.where(supratidal_elev <= HAT_interpolate, 1, np.nan)

    # HAT + storm
    HAT_storm = xr.where((supratidal_elev > HAT_interpolate) & (supratidal_elev <= HATSS_interpolate), supratidal_elev.values, np.nan)
    
    # normalise between HAT and HAT_SS
    # Find the minimum and maximum values in the data array
    min_value = HAT_storm.min()
    max_value = HAT_storm.max()
    # Normalize the data to the range [0, 1] by subtracting the minimum and dividing by the range
    HAT_storm_norm = (HAT_storm - min_value) / (max_value - min_value)
    
    # invert the normalisation and normalise between 0.5 and 1
    HAT_storm_norm_05_1 = (((1 - HAT_storm_norm)/2) + 0.5)

    # HAT + storm to 11m
    HAT_storm_11AHD = xr.where((supratidal_elev > HATSS_interpolate) & (supratidal_elev <= 11), supratidal_elev.values, np.nan)
    
    # normalise between HAT_SS and 11m AHD
    # Find the minimum and maximum values in the data array
    min_value = HAT_storm_11AHD.min()
    max_value = 11
    # Normalize the data to the range [0, 1] by subtracting the minimum and dividing by the range
    HAT_storm_11AHD_norm = (HAT_storm_11AHD - min_value) / (max_value - min_value)
    
    # invert the normalisation and normalise between 0.5 and 1
    HAT_storm_11AHD_norm_05_0 = ((1 - HAT_storm_11AHD_norm)/2)

    # combine layers back together
    supratidal_combine = ((HAT.fillna(0)) + (HAT_storm_norm_05_1.fillna(0)) + (HAT_storm_11AHD_norm_05_0.fillna(0)))
    # remove outside extent (make np.nan)
    SCF_elevation_model = xr.where(srtm_mask == 1, supratidal_combine.values, np.nan)

    ### Connectivity model ###
    # ocean nodata points
    srtm_da_buffered = srtm_buffered == srtm_buffered.nodata
    
    # combine masks for LCP, where 1 indicates aquatic features to start calculating LCP from, 0 are areas that will be given a LCP value
    aquatic_connectivity = xr.where((mangrove_buffered == True) | (saltmarsh_buffered_inter == True) | 
                                    (saltflat_buffered == True) | (CEM_buffered == True) | 
                                    (srtm_da_buffered == True), 1, 0).astype('bool') # TODO: Raf, check this fix on multiple tiles
    aquatic_connectivity = aquatic_connectivity.compute()

    # Run connectivity LCP analysis
    costdist_mask, costdist_da = load_connectivity_mask_aquatic(dc, geobox, starts_da = aquatic_connectivity)

    # mask on srtm_buffered valid values (i.e. mask out ocean from costdist_da)
    # Access nodata value from the DataArray attributes
    nodata_value = srtm_buffered.attrs.get("nodata", None)
    
    # Replace nodata values with np.nan if nodata_value is defined
    if nodata_value is not None:
        srtm_buffered_masked = srtm_buffered.where(srtm_buffered != nodata_value, np.nan)
    else:
        print("No nodata value found in attributes.")
    # Apply the mask and ensure NaNs are propagated form srtm_buffered_masked
    costdist_da_masked = costdist_da.where(~np.isnan(srtm_buffered_masked)) ### what does this do as srtm_buffered_masked may not be generated?

    min_value = 0
    max_value = 3118 #  New upper limit based on LCP for the upper Mary River field data 
    
    # Clip values above max_value
    costdist_da_clip = xr.where(costdist_da_masked > max_value, np.nan, costdist_da_masked.values)
    
    # Apply the exponential function to the pixel values of the costdist_da_clip
    # no need to invert now from last code iteration
    SCF_connectivity_model = apply_exponential(costdist_da_clip)

    ### Vegetation model ###
    # create tidal connectivity mask (TCM)
    SCF_elevation_model_mask = ~SCF_elevation_model.isnull()
    SCF_connectivity_model_mask = ~SCF_connectivity_model.isnull()
    TCM = SCF_elevation_model_mask | SCF_connectivity_model_mask

    # change to zero being where these are present, and 1 being where they are not present
    aquatic_veg_mask = xr.where((water == True) | (mangrove == True) | (saltmarsh == True) | 
                                (saltflat == True) | (CEM == True) , 0, 1).astype('bool').squeeze('time')
    aquatic_veg_mask = aquatic_veg_mask.compute()
    # TCM + veg mask
    TCM_veg_mask = TCM & aquatic_veg_mask
    
    # load woody_cover vp
    product = catalog['woody_cover']
    woody_cover_vp = product.load(dc, resampling="bilinear", **query)
    woody_cover = woody_cover_vp.woody_cover

    # remove outside extent of TCM + veg mask (make np.nan)
    SCF_veg_model = xr.where(TCM_veg_mask == 1, woody_cover.values, np.nan)

    # values where woody cover is <0.2, make these np.nan
    SCF_veg_model_02_nan = xr.where(SCF_veg_model <= 0.2, np.nan, SCF_veg_model.values)
    # values where woody cover is <0.2, make these zero
    SCF_veg_model_02_zero = xr.where(SCF_veg_model <= 0.2, 0, SCF_veg_model.values)

    # values where woody cover is <0.4, make these np.nan
    SCF_veg_model_04_nan = xr.where(SCF_veg_model <= 0.4, np.nan, SCF_veg_model.values)
    # values where woody cover is <0.4, make these np.nan
    SCF_veg_model_04_zero = xr.where(SCF_veg_model <= 0.4, 0, SCF_veg_model.values)

    # 'Re-normalise' veg model based on 0.2 or 0.4 being minimum value
    min_value_02 = 0.2
    min_value_04 = 0.4
    max_value = 1
    # 'Re-normalise' the data to the range [0, 1] by subtracting the minimum and dividing by the range
    SCF_veg_model_02_nan_renorm = (SCF_veg_model_02_nan - min_value_02) / (max_value - min_value_02)
    SCF_veg_model_04_nan_renorm = (SCF_veg_model_04_nan - min_value_04) / (max_value - min_value_04)

    # Generate supratidal and coastal floodplain forest extent model #
    # very simple approach at the moment, combine all and divide by 3 so values are within 0-1 range
    SCFF_model = ((SCF_elevation_model + SCF_connectivity_model + SCF_veg_model)/3)
    SCFF_model_02_nan = ((SCF_elevation_model + SCF_connectivity_model + SCF_veg_model_02_nan)/3)
    SCFF_model_02_zero = ((SCF_elevation_model + SCF_connectivity_model + SCF_veg_model_02_zero)/3)
    SCFF_model_02_nan_renorm = ((SCF_elevation_model + SCF_connectivity_model + SCF_veg_model_02_nan_renorm)/3)
    SCFF_model_04_nan = ((SCF_elevation_model + SCF_connectivity_model + SCF_veg_model_04_nan)/3)
    SCFF_model_04_zero = ((SCF_elevation_model + SCF_connectivity_model + SCF_veg_model_04_zero)/3)
    SCFF_model_04_nan_renorm = ((SCF_elevation_model + SCF_connectivity_model + SCF_veg_model_04_nan_renorm)/3)
    
    # output files
    SCF_elevation_model.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_elevation_model/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_elevation_model.tif', overwrite=True)
    SCF_connectivity_model.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_connectivity_model/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_connectivity_model.tif', overwrite=True)
    
    TCM_export = xr.where(TCM == 1, 1, np.nan)
    TCM_export.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/Tidal_connectivity_model/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_TCM.tif', overwrite=True)
    
    SCF_veg_model.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_veg_model/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_veg_model.tif', overwrite=True)
    SCF_veg_model_02_nan.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_veg_model_02_nan/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_veg_model_02_nan.tif', overwrite=True)
    SCF_veg_model_02_zero.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_veg_model_02_zero/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_veg_model_02_zero.tif', overwrite=True)
    SCF_veg_model_02_nan_renorm.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_veg_model_02_nan_renorm/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_veg_model_02_nan_renorm.tif', overwrite=True)
    
    SCF_veg_model_04_nan.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_veg_model_04_nan/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_veg_model_04_nan.tif', overwrite=True)
    SCF_veg_model_04_zero.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_veg_model_04_zero/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_veg_model_04_zero.tif', overwrite=True)
    SCF_veg_model_04_nan_renorm.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCF_veg_model_04_nan_renorm/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCF_veg_model_04_nan_renorm.tif', overwrite=True)
    
    SCFF_model.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCFF_model/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCFF_model.tif', overwrite=True)
    SCFF_model_02_nan.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCFF_model_02_nan/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCFF_model_02_nan.tif', overwrite=True)
    SCFF_model_02_zero.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCFF_model_02_zero/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCFF_model_02_zero.tif', overwrite=True)
    SCFF_model_02_nan_renorm.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCFF_model_02_nan_renorm/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCFF_model_02_nan_renorm.tif', overwrite=True)

    SCFF_model_04_nan.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCFF_model_04_nan/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCFF_model_04_nan.tif', overwrite=True)
    SCFF_model_04_zero.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCFF_model_04_zero/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCFF_model_04_zero.tif', overwrite=True)
    SCFF_model_04_nan_renorm.odc.write_cog('/home/jovyan/gdata1/projects/coastal/supratidal_forests/outputs/december2024_v2/SCFF_model_04_nan_renorm/' + 'x' + str(item[0]) + 'y' + str(item[1]) + '_SCFF_model_04_nan_renorm.tif', overwrite=True)



(58, 23)


/tmp/ipykernel_1750/2541330012.py:109: UserWarning: The supplied `gdf` does not overlap spatially with `ds`.
  interpolate_idw = xr_interpolate(ds=srtm, gdf=HAT_SS_gpd, columns=['HAT', 'SSElev'], method="idw")
/env/lib/python3.10/site-packages/datacube/virtual/transformations.py:440: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  return time.astype('datetime64[Y]')
/env/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


(59, 23)


/tmp/ipykernel_1750/2541330012.py:109: UserWarning: The supplied `gdf` does not overlap spatially with `ds`.
  interpolate_idw = xr_interpolate(ds=srtm, gdf=HAT_SS_gpd, columns=['HAT', 'SSElev'], method="idw")
/env/lib/python3.10/site-packages/rasterio/warp.py:387: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dest = _reproject(


In [13]:
end = time.time()
elapsed_time = end - start
minutes = int(elapsed_time // 60)
seconds = elapsed_time % 60
print(f"Elapsed time: {minutes} minutes and {seconds:.2f} seconds")

Elapsed time: 7 minutes and 17.30 seconds


In [15]:
# want to output model as

# SCFF_model - three models combined X

# SCFF_model_02_nan - threshold WCF model by 0.2 to be np.nan X
# SCFF_model_04_nan - threshold WCF model by 0.4 to be np.nan X

# SCF_veg_model_02_zero - threshold WCF model by 0.2 to be zero x
# SCF_veg_model_04_zero - threshold WCF model by 0.4 to be zero x

# SCFF_model_02_nan_norm - threshold WCF model by 0.2 to be np.nan, then renormalise values to 0-1 x
# SCFF_model_04_nan_norm - threshold WCF model by 0.4 to be np.nan, then renormalise values to 0-1 x